In [119]:
import pykep as pk
import numpy as np
pk.util.load_spice_kernel("/home/m/mmk40/Desktop/Planet Position/de431.bsp")
from pykep.orbit_plots import plot_planet, plot_lambert
from pykep import AU, DAY2SEC, SEC2DAY, DAY2YEAR
import pygmo as pg
from datetime import datetime, timedelta
import math

In [120]:
# Load planets
venus = pk.planet.spice('VENUS' , 'SUN', 'ECLIPJ2000', 'NONE')
jupiter = pk.planet.spice('JUPITER BARYCENTER', 'SUN', 'ECLIPJ2000', 'NONE' )

# Set launch window
start_date = pk.epoch_from_string("2039-Oct-17 00:00:00")
end_date = pk.epoch_from_string("2046-01-01 00:00:00")

# Parameters
launch_step = 30     # days between launch trials
tof_range = (300, 1500)  # in days, time of flight range

In [121]:
# Results list
results = []

# Scan through departure dates
t_dep = start_date
while t_dep.mjd2000 < end_date.mjd2000:
    rV, vV = venus.eph(t_dep)

    for tof_days in range(tof_range[0], tof_range[1], 30):  # try multiple TOFs
        try:
            t_arr = pk.epoch(t_dep.mjd2000 + tof_days)
            rJ, vJ = jupiter.eph(t_arr)

            dt_sec = tof_days * pk.DAY2SEC
            l = pk.lambert_problem(r1=rV, r2=rJ, tof=dt_sec, mu=pk.MU_SUN, max_revs=4)

            v1 = np.array(l.get_v1()[0])
            v2 = np.array(l.get_v2()[0])
            v_inf = (v1 - np.array(vV))/1000
            delta_v = (np.linalg.norm(v1 - np.array(vV)))/1000 
            v_arrival   = l.get_v2()[0] # velocity of spacecraft at jupiter arrival
            hyperbolic_v_arrival   = (np.linalg.norm(np.array(v_arrival) - np.array(vJ)))/1000   # km/s  # substracting velocities and taking magnitude
            C3 = np.linalg.norm(v_inf) ** 2  # in km²/s²

            # Calculating Flyby speed

            rp = 9 * 69911e3 # m  # radius of periapsis for the flyby trajectory (change this)

            mu = 6.67e-11 * 1.898e27 # m3/s2  # gravitationalparameter for jupiter

            v_in = hyperbolic_v_arrival * 1000 # m/s # Jupiter arrival speed

            # Turning Angle
            phi = np.degrees(np.pi - 2 * np.acos (1/(((v_in**2)*rp)/mu + 1)))
            # print (np.pi) # Degree/radian check
            # print (f" Turning angle: {phi:.2f} degrees") # degrees


            # Law of Cosines to find the v_out
            #print(vJ) # vector/scalar check
            vJ_mag = np.linalg.norm(vJ)
            #print(vJ_mag)

            exit_speed = (math.sqrt((vJ_mag**2) + (v_in**2) - 2*vJ_mag*v_in*np.cos(phi)))/1000 # km/s
            #print(f"Flyby speed: {(exit_speed - hyperbolic_v_arrival):.2f} km/s")
            #print (f"Jupiter Departure Speed: {exit_speed:.2f} km/s")

            # Time to Heliopause from Jupiter flyby

            d = 201.44*AU # 200 AU in meters
            s = exit_speed*1000 # speed of space craft at jupiter exit in m/s 

            # print (d) .pk check

            time_to_200AU = (d/s)*SEC2DAY*DAY2YEAR
            total_time = time_to_200AU + tof_days/365
            flyby_speed = exit_speed - hyperbolic_v_arrival

            #print(f"Time to reach Jupiter: {dt*SEC2DAY*DAY2YEAR: .2f} years")
            #print(f"Time to 200 AU from Jupiter Flyby: {time_to_200AU:.2f} years")
            #print(f"Total Time to reach 200 AU from launch: {time_to_200AU + dt*SEC2DAY*DAY2YEAR : .2f} years")

            d1 = 91.44*AU
            s1 = exit_speed*1000

            time_to_90AU = (d1/s1)*SEC2DAY*DAY2YEAR # time to reach 90 AU

            d2 = 121.44*AU
            s2 = exit_speed*1000

            time_to_120AU = (d2/s2)*SEC2DAY*DAY2YEAR # time to reach 120 AU

            

            
            
            if C3 < 140:
                if total_time < 70:

                    # Get Keplerian elements of transfer orbit
                    kep = pk.ic2par(rV, v1, pk.MU_SUN)  # r, v at arrival → orbital elements

                    results.append({
                        'departure': str(t_dep),
                        'arrival': str(t_arr),
                        'tof_days': tof_days,
                        'C3': C3,
                        'delta_v': delta_v,
                        'sma': kep[0],   # semi-major axis
                        'ecc': kep[1],   # eccentricity
                        'inc': np.degrees(kep[2]),  # inclination (rad → deg)
                        'raan': np.degrees(kep[3]),
                        'argp': np.degrees(kep[4]),
                        'ta': np.degrees(kep[5]),
                        'turning angle': phi,
                        'flyby speed': flyby_speed,
                        'venus departure speed': exit_speed,
                        'time_to_90AU': time_to_90AU,
                        'time_to_120AU': time_to_120AU,
                        'jupiter flyby to 200 au': time_to_200AU,
                        'total_time' : total_time, 
                    
                        })

        except RuntimeError as e:
            # Lambert solver may fail for some configurations; skip them
            continue

    t_dep = pk.epoch(t_dep.mjd2000 + launch_step)

In [122]:
# Sort by C3 (ascending)
results.sort(key=lambda x: x['total_time'])

# Print trajectories
for res in results[:]:
    print(f"Launch: {res['departure']}, Arrival: {res['arrival']}, C3: {res['C3']:.2f} km²/s², Delta v: {res['delta_v']:.2f} km/s \n\nTime to reach Jupiter: {res['tof_days']/365:.2f} years, Time to 200 AU from Jupiter Flyby: {res['jupiter flyby to 200 au']: .2f} years, Total Time to 200 AU from Launch: {res['total_time']:.2f} years\n")
    print(f"Time to reach 90 AU: {res['time_to_90AU']: .2f} years, Time to reach 120 AU: {res['time_to_120AU']: .2f} years\n")
    print(f"Turning angle: {res['turning angle']: .2f}° | Flyby Speed: {res['flyby speed']: .2f} km/s \n ")
    print(f"Elements for VENUS -> JUPITER → SMA: {res['sma']/1000:.3f} km, ECC: {res['ecc']:.4f}, INC: {res['inc']:.2f}°\n")
    print(f"Elements VENUS -> JUPITER → RAAN: {res['raan']:.3f}° , AP: {res['argp']:.4f}°, TA: {res['ta']:.2f}°\n\n\n")

Launch: 2044-Dec-19 00:00:00, Arrival: 2047-Nov-04 00:00:00, C3: 136.93 km²/s², Delta v: 11.70 km/s 

Time to reach Jupiter: 2.88 years, Time to 200 AU from Jupiter Flyby:  45.56 years, Total Time to 200 AU from Launch: 48.44 years

Time to reach 90 AU:  20.68 years, Time to reach 120 AU:  27.47 years

Turning angle:  103.78° | Flyby Speed:  13.58 km/s 
 
Elements for VENUS -> JUPITER → SMA: 433941790.548 km, ECC: 0.7518, INC: 5.33°

Elements VENUS -> JUPITER → RAAN: 60.106° , AP: 162.7078°, TA: -2.83°



Launch: 2044-May-23 00:00:00, Arrival: 2048-Jun-01 00:00:00, C3: 139.79 km²/s², Delta v: 11.82 km/s 

Time to reach Jupiter: 4.03 years, Time to 200 AU from Jupiter Flyby:  45.08 years, Total Time to 200 AU from Launch: 49.11 years

Time to reach 90 AU:  20.46 years, Time to reach 120 AU:  27.18 years

Turning angle:  90.39° | Flyby Speed:  12.11 km/s 
 
Elements for VENUS -> JUPITER → SMA: 480147861.852 km, ECC: 0.7749, INC: 2.26°

Elements VENUS -> JUPITER → RAAN: 86.039° , AP: 146.